In [1]:
# Instala as dependências necessárias
!pip install polars plotly gdown


In [2]:
# Baixa o arquivo Parquet do Google Drive usando gdown
# O arquivo está disponível em: https://drive.google.com/file/d/1Rwp6DkqNewtR6WYstAKypXAsITGPFhvj/view?usp=sharing

!gdown --id 1Rwp6DkqNewtR6WYstAKypXAsITGPFhvj -O BancoVDE_2015_2025.parquet


/usr/local/lib/python3.11/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1Rwp6DkqNewtR6WYstAKypXAsITGPFhvj
To: /content/BancoVDE_2015_2025.parquet
100% 40.9M/40.9M [00:00<00:00, 50.0MB/s]


In [10]:
import polars as pl

# Carrega o Parquet em modo eager para inspecionar
df_lazy = pl.read_parquet("BancoVDE_2015_2025.parquet")

# Exibir as primeiras linhas
print(df_lazy.head())

# Exibir os tipos das colunas
print(df_lazy.schema)


shape: (5, 16)
┌─────┬─────────────┬─────────────┬─────────────┬───┬────────────┬─────────────┬────────────┬──────┐
│ uf  ┆ municipio   ┆ evento      ┆ data_refere ┆ … ┆ total_peso ┆ abrangencia ┆ formulario ┆ ano  │
│ --- ┆ ---         ┆ ---         ┆ ncia        ┆   ┆ ---        ┆ ---         ┆ ---        ┆ ---  │
│ str ┆ str         ┆ str         ┆ ---         ┆   ┆ f64        ┆ str         ┆ str        ┆ i32  │
│     ┆             ┆             ┆ str         ┆   ┆            ┆             ┆            ┆      │
╞═════╪═════════════╪═════════════╪═════════════╪═══╪════════════╪═════════════╪════════════╪══════╡
│ AC  ┆ NÃO         ┆ Apreensão   ┆ 01-01-15    ┆ … ┆ null       ┆ Estadual    ┆ Formulário ┆ 2015 │
│     ┆ INFORMADO   ┆ de Cocaína  ┆             ┆   ┆            ┆             ┆ 5          ┆      │
│ AL  ┆ NÃO         ┆ Apreensão   ┆ 01-01-15    ┆ … ┆ 19.89      ┆ Estadual    ┆ Formulário ┆ 2015 │
│     ┆ INFORMADO   ┆ de Cocaína  ┆             ┆   ┆            ┆          

In [20]:
import polars as pl

# Define os eventos desejados
eventos = ["Feminicídio", "Tentativa de feminicídio", "Tentativa de homicídio", "Homicídio doloso"]

# Filtra os registros onde o evento esteja na lista
df_lazy = df_lazy.filter(pl.col("evento").is_in(eventos))

# Converte a coluna 'data_referencia' (formato "dd-mm-yy") para tipo Date, com strict=False para valores incompatíveis
df_lazy = df_lazy.with_columns(
    pl.col("data_referencia").str.strptime(pl.Date, format="%d-%m-%y", strict=False).alias("data_referencia_date")
)

# Converte a coluna "feminino" para float para permitir a soma
df_lazy = df_lazy.with_columns(
    pl.col("feminino").cast(pl.Float64)
)

# Cria uma coluna 'year_month' truncando a data para o primeiro dia do mês
df_lazy = df_lazy.with_columns(
    pl.col("data_referencia_date").dt.truncate("1mo").alias("year_month")
)

# Agrega por UF e Mês, somando os valores da coluna "feminino"
agg_lazy = df_lazy.group_by(["uf", "year_month"]).agg(
    pl.col("feminino").sum().alias("feminino_sum")
).sort(["uf", "year_month"])

# Executa a computação
df_agg = agg_lazy.collect()

# Exibe as primeiras linhas da base agregada
print(df_agg.head())

# Exibe os tipos das colunas na base agregada
print(df_agg.schema)


shape: (5, 3)
┌─────┬────────────┬──────────────┐
│ uf  ┆ year_month ┆ feminino_sum │
│ --- ┆ ---        ┆ ---          │
│ str ┆ date       ┆ f64          │
╞═════╪════════════╪══════════════╡
│ AC  ┆ 2015-01-01 ┆ 16.0         │
│ AC  ┆ 2016-01-01 ┆ 25.0         │
│ AC  ┆ 2017-01-01 ┆ 33.0         │
│ AC  ┆ 2018-01-01 ┆ 49.0         │
│ AC  ┆ 2019-01-01 ┆ 82.0         │
└─────┴────────────┴──────────────┘
Schema([('uf', String), ('year_month', Date), ('feminino_sum', Float64)])


In [21]:
import plotly.express as px

# Lista de estados (UFs) únicos
ufs = df_agg.select("uf").unique().to_series().to_list()

# Gera um gráfico para cada UF
for uf in ufs:
    df_uf = df_agg.filter(pl.col("uf") == uf).sort("year_month").to_pandas()

    fig = px.line(
        df_uf,
        x="year_month",
        y="feminino_sum",  # Alterado para feminino_sum
        title=f"Série Temporal de Feminicídios - {uf}",
        labels={"year_month": "Mês", "feminino_sum": "Total Feminino"}
    )
    fig.update_layout(xaxis=dict(dtick="M3", tickformat="%b\n%Y"))
    fig.show()
